In [2]:
!ls data/in/logger_data/2024.12.16/

can1.csv  can2.csv  oil1.csv  oil2.csv	oil3.csv


In [2]:
import pandas as pd
import os
import glob
import re

def get_final_can_df(folder_date_str):
    """
    Given a folder name like '2024.12.16', returns the final CAN dataframe
    with unwanted columns dropped and grouped by floored datetime.
    """
    
    folder_path = os.path.join('data/in/logger_data/', folder_date_str)
    can_files = glob.glob(os.path.join(folder_path, 'can*.csv'))
    cols_to_drop = ['AFR', 'AFR-LC', 'ADC_MAF', 'SSM', 'ACC_ENGINE']
    all_dfs = []

    for file in can_files:
        df = pd.read_csv(file, delimiter=';', decimal=',')
        df = df[df['RPM'] != 0]
        df['datetime_temp'] = pd.to_datetime(
            folder_date_str + ' ' + df['Time'].astype(str),
            format='%Y.%m.%d %H:%M:%S.%f',
            errors='coerce'
        )
        df['datetime'] = df['datetime_temp'].dt.floor('s')
        df_grouped = df.groupby('datetime').median(numeric_only=True).reset_index()
        df_grouped = df_grouped.drop(columns=[col for col in cols_to_drop if col in df_grouped.columns])
        all_dfs.append(df_grouped)

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df = final_df.sort_values(by='datetime').reset_index(drop=True)
        return final_df
    else:
        print(f"No CAN files found in {folder_path}")
        return pd.DataFrame()

def get_final_oil_df(folder_date_str):
    """
    Given a folder name like '2024.12.16', returns the final OIL dataframe
    with the first 5 columns, cleaned and grouped by floored datetime.
    """

    folder_path = os.path.join('data/in/logger_data/', folder_date_str)
    oil_files = glob.glob(os.path.join(folder_path, 'oil*.csv'))
    oil_dfs = []

    for file in oil_files:
        print(f"Processing file: {file}")
        df = pd.read_csv(file, delimiter=';', decimal=',')
        df = df.iloc[:, :5]  # Take only the first 5 columns
        # Clean numeric columns
        for col in ['vp', 'vt']:
            df[col] = df[col].astype(str).str.replace(r'[^0-9,.-]', '', regex=True)
            df[col] = df[col].str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna()
        # Combine folder date with 'Time' column to create full datetime
        df['datetime'] = pd.to_datetime(
            folder_date_str + ' ' + df['Time'].astype(str),
            format='%Y.%m.%d %H:%M:%S',
            errors='coerce'
        )
        df_grouped = df.groupby('datetime').median(numeric_only=True).reset_index()
        oil_dfs.append(df_grouped)
    
    if oil_dfs:
        oil_final_df = pd.concat(oil_dfs, ignore_index=True)
        oil_final_df = oil_final_df.sort_values(by='datetime').reset_index(drop=True)
        return oil_final_df
    else:
        print(f"No OIL files found in {folder_path}")
        return pd.DataFrame()

In [11]:
parent_folder = 'data/in/logger_data/'
folder_names = [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d))]

all_merged_dfs = []

for folder_name in folder_names:
    print(f"Processing folder: {folder_name}")
    can_final_df = get_final_can_df(folder_name)
    oil_final_df = get_final_oil_df(folder_name)
    merged_df = pd.merge(can_final_df, oil_final_df, left_on='datetime', right_on='datetime', how='inner')
    print(f"can_final_df shape for {folder_name}: {can_final_df.shape}")
    print(f"oil_final_df shape for {folder_name}: {oil_final_df.shape}")
    print(f"merged_df shape for {folder_name}: {merged_df.shape}")
    # Save merged CSV inside the same folder
    output_path = os.path.join(parent_folder, folder_name, f'merged_{folder_name}.csv')
    merged_df.to_csv(output_path, index=False)
    # Store for final concatenation
    all_merged_dfs.append(merged_df)

# Concatenate all merged dataframes into one final dataframe
if all_merged_dfs:
    final_merged_df = pd.concat(all_merged_dfs, ignore_index=True)
    final_merged_df = final_merged_df.sort_values(by='datetime').reset_index(drop=True)
    final_merged_df.to_csv(os.path.join(parent_folder, 'all_merged_data.csv'), 
                           index=False, float_format='%.2f')
    print(f"Final merged dataframe shape: {final_merged_df.shape}")
else:
    print("No merged dataframes to concatenate.")

Processing folder: 2025.02.02
Processing file: data/in/logger_data/2025.02.02/oil2.csv
Processing file: data/in/logger_data/2025.02.02/oil3.csv
Processing file: data/in/logger_data/2025.02.02/oil4.csv
Processing file: data/in/logger_data/2025.02.02/oil1.csv
can_final_df shape for 2025.02.02: (6340, 18)
oil_final_df shape for 2025.02.02: (6133, 5)
merged_df shape for 2025.02.02: (6095, 22)
Processing folder: 2024.12.30
Processing file: data/in/logger_data/2024.12.30/oil2.csv
Processing file: data/in/logger_data/2024.12.30/oil3.csv
Processing file: data/in/logger_data/2024.12.30/oil1.csv
can_final_df shape for 2024.12.30: (9793, 18)
oil_final_df shape for 2024.12.30: (6229, 5)
merged_df shape for 2024.12.30: (6155, 22)
Processing folder: 2024.12.28
Processing file: data/in/logger_data/2024.12.28/oil2.csv
Processing file: data/in/logger_data/2024.12.28/oil1.csv
can_final_df shape for 2024.12.28: (6910, 18)
oil_final_df shape for 2024.12.28: (8165, 5)
merged_df shape for 2024.12.28: (6868,

/tmp/ipykernel_107505/4089064942.py:50: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter=';', decimal=',')


can_final_df shape for 2024.12.21: (17179, 18)
oil_final_df shape for 2024.12.21: (17325, 5)
merged_df shape for 2024.12.21: (17179, 22)
Processing folder: 2025.01.05
Processing file: data/in/logger_data/2025.01.05/oil2.csv
Processing file: data/in/logger_data/2025.01.05/oil3.csv
Processing file: data/in/logger_data/2025.01.05/oil1.csv
can_final_df shape for 2025.01.05: (6722, 18)
oil_final_df shape for 2025.01.05: (8019, 5)
merged_df shape for 2025.01.05: (6659, 22)
Processing folder: 2025.01.20
Processing file: data/in/logger_data/2025.01.20/oil2.csv
Processing file: data/in/logger_data/2025.01.20/oil3.csv
Processing file: data/in/logger_data/2025.01.20/oil4.csv
Processing file: data/in/logger_data/2025.01.20/oil1.csv
Processing file: data/in/logger_data/2025.01.20/oil5.csv
can_final_df shape for 2025.01.20: (8230, 18)
oil_final_df shape for 2025.01.20: (9962, 5)
merged_df shape for 2025.01.20: (8129, 22)
Processing folder: 2025.01.04
Processing file: data/in/logger_data/2025.01.04/o

/tmp/ipykernel_107505/4089064942.py:50: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter=';', decimal=',')


Processing file: data/in/logger_data/2024.12.20/oil5.csv
can_final_df shape for 2024.12.20: (19694, 18)
oil_final_df shape for 2024.12.20: (22676, 5)
merged_df shape for 2024.12.20: (19631, 22)
Processing folder: 2025.02.04
Processing file: data/in/logger_data/2025.02.04/oil2.csv
Processing file: data/in/logger_data/2025.02.04/oil3.csv
Processing file: data/in/logger_data/2025.02.04/oil1.csv
can_final_df shape for 2025.02.04: (11503, 18)
oil_final_df shape for 2025.02.04: (11436, 5)
merged_df shape for 2025.02.04: (11406, 22)
Processing folder: 2025.02.12
Processing file: data/in/logger_data/2025.02.12/oil2.csv
Processing file: data/in/logger_data/2025.02.12/oil3.csv
Processing file: data/in/logger_data/2025.02.12/oil1.csv
can_final_df shape for 2025.02.12: (9890, 18)
oil_final_df shape for 2025.02.12: (9942, 5)
merged_df shape for 2025.02.12: (9832, 22)
Processing folder: 2025.02.05
Processing file: data/in/logger_data/2025.02.05/oil2.csv
Processing file: data/in/logger_data/2025.02.0

...

Processing file: data/in/logger_data/2024.12.24/oil1.csv
Processing file: data/in/logger_data/2024.12.24/oil7.csv
Processing file: data/in/logger_data/2024.12.24/oil5.csv
can_final_df shape for 2024.12.24: (31561, 18)
oil_final_df shape for 2024.12.24: (33337, 5)
merged_df shape for 2024.12.24: (31415, 22)
Processing folder: 2025.02.11
Processing file: data/in/logger_data/2025.02.11/oil2.csv
Processing file: data/in/logger_data/2025.02.11/oil3.csv
Processing file: data/in/logger_data/2025.02.11/oil1.csv
can_final_df shape for 2025.02.11: (9051, 18)
oil_final_df shape for 2025.02.11: (7361, 5)
merged_df shape for 2025.02.11: (7290, 22)
Processing folder: 2025.01.11
Processing file: data/in/logger_data/2025.01.11/oil2.csv
Processing file: data/in/logger_data/2025.01.11/oil3.csv
Processing file: data/in/logger_data/2025.01.11/oil6.csv
Processing file: data/in/logger_data/2025.01.11/oil4.csv
Processing file: data/in/logger_data/2025.01.11/oil1.csv
Processing file: data/in/logger_data/2025.

/tmp/ipykernel_107505/4089064942.py:50: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter=';', decimal=',')


Processing folder: 2025.01.30
Processing file: data/in/logger_data/2025.01.30/oil2.csv
Processing file: data/in/logger_data/2025.01.30/oil3.csv
Processing file: data/in/logger_data/2025.01.30/oil6.csv
Processing file: data/in/logger_data/2025.01.30/oil4.csv
Processing file: data/in/logger_data/2025.01.30/oil1.csv
Processing file: data/in/logger_data/2025.01.30/oil5.csv
can_final_df shape for 2025.01.30: (15641, 18)
oil_final_df shape for 2025.01.30: (16055, 5)
merged_df shape for 2025.01.30: (15456, 22)
Processing folder: 2025.02.07
Processing file: data/in/logger_data/2025.02.07/oil2.csv
Processing file: data/in/logger_data/2025.02.07/oil3.csv
Processing file: data/in/logger_data/2025.02.07/oil4.csv
Processing file: data/in/logger_data/2025.02.07/oil1.csv


/tmp/ipykernel_107505/4089064942.py:50: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter=';', decimal=',')


Processing file: data/in/logger_data/2025.02.07/oil5.csv
can_final_df shape for 2025.02.07: (25292, 18)
oil_final_df shape for 2025.02.07: (25444, 5)
merged_df shape for 2025.02.07: (24088, 22)
Processing folder: 2024.12.18
Processing file: data/in/logger_data/2024.12.18/oil2.csv
Processing file: data/in/logger_data/2024.12.18/oil3.csv
Processing file: data/in/logger_data/2024.12.18/oil6.csv
Processing file: data/in/logger_data/2024.12.18/oil4.csv
Processing file: data/in/logger_data/2024.12.18/oil1.csv
Processing file: data/in/logger_data/2024.12.18/oil8.csv
Processing file: data/in/logger_data/2024.12.18/oil7.csv
Processing file: data/in/logger_data/2024.12.18/oil5.csv
can_final_df shape for 2024.12.18: (38886, 18)
oil_final_df shape for 2024.12.18: (39999, 5)
merged_df shape for 2024.12.18: (38733, 22)
Final merged dataframe shape: (473799, 22)


In [12]:
# Check if 'final_merged_df' is sorted by 'datetime'
is_sorted = final_merged_df['datetime'].is_monotonic_increasing
print("Is final_merged_df sorted by datetime?", is_sorted)

Is final_merged_df sorted by datetime? True


In [13]:
final_merged_df.shape

(473799, 22)

### Adding some features to dataset

In [2]:
import pandas as pd

In [15]:
df = pd.read_csv('all_merged_data.csv', parse_dates=['datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473799 entries, 0 to 473798
Data columns (total 22 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   datetime   473799 non-null  datetime64[ns]
 1   RPM        473798 non-null  float64       
 2   THR        473798 non-null  float64       
 3   Fdet       473798 non-null  float64       
 4   Speed      473798 non-null  float64       
 5   Twat       473798 non-null  float64       
 6   INJ        473798 non-null  float64       
 7   AIR        473798 non-null  float64       
 8   GBC        473798 non-null  float64       
 9   ADC_LAM    473798 non-null  float64       
 10  UOZ        473798 non-null  float64       
 11  Press      473797 non-null  float64       
 12  COEFF      473797 non-null  float64       
 13  Tair       473796 non-null  float64       
 14  Fxx        473796 non-null  float64       
 15  Ubat       473796 non-null  float64       
 16  TPS        473796 no

In [16]:
import numpy as np

gap_threshold = pd.Timedelta(seconds=50)
time_diff = df['datetime'].diff()
df['trip_number'] = (time_diff > gap_threshold).cumsum() + 1
print(df[['datetime', 'trip_number']].head(20))

              datetime  trip_number
0  2024-12-16 16:13:59            1
1  2024-12-16 16:14:00            1
2  2024-12-16 16:14:01            1
3  2024-12-16 16:14:02            1
4  2024-12-16 16:14:03            1
5  2024-12-16 16:14:04            1
6  2024-12-16 16:14:05            1
7  2024-12-16 16:14:06            1
8  2024-12-16 16:14:07            1
9  2024-12-16 16:14:08            1
10 2024-12-16 16:14:09            1
11 2024-12-16 16:14:10            1
12 2024-12-16 16:14:11            1
13 2024-12-16 16:14:12            1
14 2024-12-16 16:14:13            1
15 2024-12-16 16:14:14            1
16 2024-12-16 16:14:15            1
17 2024-12-16 16:14:16            1
18 2024-12-16 16:14:17            1
19 2024-12-16 16:14:18            1


In [17]:
df.to_csv('all_merged_data.csv', index=False, float_format='%.2f')